### Regis University

**MSDS688_X70: Artificial Intelligence**  
Master of Science in Data Science Program


#### Week 2: Reinforcement Learning for AI Agents


## Lecture: Week 2 - Epsilon Decay and Reinforcement Learning Strategies

### Overview

In this week’s assignment, we will focus on an important concept in **reinforcement learning (RL)**: **Epsilon Decay**. This is especially relevant for exploration in RL algorithms like **Q-Learning** and **Deep Q Networks (DQN)**. The goal is to understand the role of epsilon decay in managing the exploration-exploitation trade-off.

---

### 1. **Introduction to Reinforcement Learning**

Reinforcement learning is a type of machine learning where an **agent** learns by interacting with an environment. The agent takes actions to maximize cumulative rewards. It involves the following key components:
- **Agent**: The learner or decision maker.
- **Environment**: The world through which the agent interacts.
- **State (s)**: The current situation of the agent.
- **Action (a)**: The decisions the agent makes.
- **Reward (r)**: The feedback from the environment after an action.

The agent aims to learn an **optimal policy** that maximizes the total reward over time by balancing two conflicting strategies:
- **Exploration**: Trying new actions to discover more information about the environment.
- **Exploitation**: Using known actions that give the highest reward based on the current knowledge.

---

### 2. **Epsilon-Greedy Strategy**

In RL, one common approach to balance exploration and exploitation is the **epsilon-greedy** strategy:
- With probability **epsilon** (𝜖), the agent chooses a random action (exploration).
- With probability **1 - epsilon**, the agent chooses the action that maximizes the known reward (exploitation).

#### How Epsilon Decay Works:
- **Initial Exploration**: At the start, **epsilon** is usually set to a high value (close to 1), so the agent explores a lot of different actions.
- **Decay Over Time**: As the agent learns more about the environment, **epsilon** gradually decreases, encouraging the agent to exploit the best-known actions more frequently.

#### Diagram: Epsilon Decay

```
Initial Epsilon (𝜖 = 1)               Final Epsilon (𝜖 = 0.1)
-------------------|--------------------------------------|-->
   Exploration                       Exploitation
```

---

### 3. **Epsilon Decay Formula**

A common method to control the decay of epsilon over episodes is using the **exponential decay function**:

epsilon = epsilon_min + (epsilon_max - epsilon_min) * exp(-lambda * episode)

Where:

- epsilon_min: The minimum value epsilon will reach after sufficient episodes.
- epsilon_max: The initial value of epsilon at the beginning of training.
- lambda: The decay rate that controls how fast epsilon decreases.


#### Key Parameters:
- **Episode**: A single run of the agent through the environment.
- **Epsilon**: The exploration probability.

---

### 4. **Exploration vs. Exploitation Trade-Off**

The **exploration-exploitation trade-off** is central to reinforcement learning:
- **Exploration**: Helps the agent discover new, potentially better actions.
- **Exploitation**: Allows the agent to leverage what it has already learned to maximize rewards.

Balancing these two is critical:
- **Too much exploration**: The agent may waste time on suboptimal actions.
- **Too much exploitation**: The agent may get stuck in local optima and fail to discover better strategies.

Epsilon decay helps the agent gradually shift from exploration (early in training) to exploitation (as the policy becomes more refined).

---

### 5. **CartPole Environment**
Before diving into the **MountainCar** environment, let's explore the **CartPole** environment, another classic problem in reinforcement learning. In CartPole:

- The agent controls a cart that can move left or right on a one-dimensional track.
- A pole is attached to the cart by an un-actuated joint. The goal is to prevent the pole from falling over by balancing it upright.
- The agent receives a reward for each time step that the pole remains upright.

CartPole is an excellent environment for understanding basic reinforcement learning concepts because it provides quick feedback and demonstrates the importance of balancing exploration and exploitation. Using the epsilon-greedy strategy with epsilon decay can significantly impact the agent's ability to learn a successful balancing policy.

---

### 6. **MountainCar Environment**

In this assignment, you are working with the **MountainCar** environment, which is a standard problem in reinforcement learning. In this environment:
- The goal is for the agent (a car) to reach the top of a hill.
- The agent must learn to balance momentum and use exploration to figure out how to build enough velocity to climb the hill.

The challenge in MountainCar is that the agent must explore enough actions (like driving backward before moving forward) to find an optimal strategy. This makes **epsilon-greedy** and **epsilon decay** crucial for solving the problem efficiently.

---

### 7. **Visualizing Epsilon Decay**

In this assignment, you will visualize how epsilon decreases over episodes. The graph provides insight into how quickly the agent transitions from exploration to exploitation.

#### Example Graph:

```
     Epsilon Decay Over Episodes
      +-----------------------------------------+
      |                                         |
      |              __________                 |
      |             /          \                |
      |            /            \               |
      |___________/              \______________|
      |                                         |
      +-----------------------------------------+
      Episodes ---->                     Epsilon Value
```

You will implement the **epsilon decay function** and track the epsilon values over time, plotting the resulting curve.

---

### Conclusion

This week’s assignment focuses on understanding how **epsilon decay** affects learning in reinforcement learning. The exploration-exploitation trade-off is fundamental to making efficient decisions in an uncertain environment. By tuning the decay rate and monitoring epsilon over episodes, we can achieve a balance between discovering new actions and exploiting known good strategies.


## Assignment Part 1: Follow Me – Q-learning in CartPole

In this section, you will apply Q-learning to the CartPole environment and visualize the total rewards over episodes. The goal is to solve the classic control problem where a cart must balance a pole upright using reinforcement learning. Through Q-learning, the agent will learn to take appropriate actions to keep the pole balanced for as long as possible.


In [ ]:
!pip install gymnasium matplotlib

In [ ]:
import gymnasium as gym
import numpy as np
import random
import math
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output


In [ ]:
# Create the CartPole environment
env = gym.make('CartPole-v1')
state = env.reset()[0]  # Reset returns a tuple with (observation, info)

In [ ]:
# Discretization parameters: one bucket for each of the four dimensions
num_buckets = (6, 6, 12, 12)  # (cart position, cart velocity, pole angle, pole velocity at tip)
state_bounds = list(zip(env.observation_space.low, env.observation_space.high))

In [ ]:
# Clip the bounds to avoid infinite values for CartPole
state_bounds[1] = [-4.8, 4.8]  # Cart velocity bounds
state_bounds[3] = [-math.radians(50), math.radians(50)]  # Pole velocity bounds

In [ ]:
# Q-table size: depends on discretization
q_table = np.zeros(num_buckets + (env.action_space.n,))

In [ ]:
# Hyperparameters
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995  # Decay rate for exploration
min_epsilon = 0.01  # Minimum exploration rate
episodes = 1000  # Number of episodes to train

In [ ]:
episode_rewards = []
epsilon_values = []

In [ ]:
# Function to discretize the continuous state
def discretize_state(state):
    discrete_state = []
    for i in range(len(state)):
        scaling = (state[i] - state_bounds[i][0]) / (state_bounds[i][1] - state_bounds[i][0])
        new_state = int(scaling * num_buckets[i])
        new_state = min(num_buckets[i] - 1, max(0, new_state))  # Clip state within bucket range
        discrete_state.append(new_state)
    return tuple(discrete_state)

In [ ]:
# Q-Learning Algorithm for CartPole
for episode in range(episodes):
    state = env.reset()[0]  # Grab the observation from reset
    state = discretize_state(state)
    done = False
    total_reward = 0

    while not done:
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample()  # Explore action space
        else:
            action = np.argmax(q_table[state])  # Exploit learned values

        # Take action, observe new state, reward, done (terminated), truncated, and info
        next_state, reward, done, truncated, info = env.step(action)
        next_state = discretize_state(next_state)  # Discretize next state

        # Update Q-table
        q_table[state][action] = q_table[state][action] + alpha * (
                reward + gamma * np.max(q_table[next_state]) - q_table[state][action])

        # Update state
        state = next_state
        total_reward += reward

        # Check if episode is done or truncated
        if done or truncated:
            break

    # Append total reward for each episode
    episode_rewards.append(total_reward)

    # Append current epsilon value
    epsilon_values.append(epsilon)

    # Decay epsilon to reduce exploration over time
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    # Print results for every 100 episodes
    if episode % 100 == 0:
        print(f"Episode {episode} - Total Reward: {total_reward}")

In [ ]:
print("Training completed for CartPole!")

Visualization of the results for CartPole

In [ ]:
plt.figure(figsize=(12, 5))

In [ ]:
# Total Reward per Episode
plt.subplot(1, 2, 1)
plt.plot(episode_rewards, color='blue')
plt.title('Total Reward per Episode (CartPole)')
plt.xlabel('Episode')
plt.ylabel('Total Reward')

In [ ]:
# Epsilon Decay over Episodes
plt.subplot(1, 2, 2)
plt.plot(epsilon_values, color='green')
plt.title('Epsilon Decay over Episodes (CartPole)')
plt.xlabel('Episode')
plt.ylabel('Epsilon Value')

In [ ]:
plt.tight_layout()
plt.show()

## Assignment Part 2: Your Turn – Q-learning in MountainCar
In this section, you will apply Q-learning to the MountainCar-v0 environment and visualize the total rewards over episodes. The goal is to solve the classic control problem where a car must navigate up a steep hill using reinforcement learning. **A framework has been provided, and your job is to complete the TODOs.**

In [ ]:
!pip install imageio

In [ ]:
# Create the MountainCar environment
env = gym.make('MountainCar-v0', render_mode="rgb_array")
state = env.reset()[0]  # Reset returns a tuple with (observation, info)

In [ ]:
# Discretization parameters for MountainCar
num_buckets = (18, 14)  # (car position, car velocity)
state_bounds = list(zip(env.observation_space.low, env.observation_space.high))

In [ ]:
# Q-table size: depends on discretization
q_table = np.zeros(num_buckets + (env.action_space.n,))

In [ ]:
# Adjust hyperparameters for MountainCar
##### TODO: Customize alpha, gamma, and epsilon for experimentation ####
alpha = 0.1  # Learning rate
gamma = 0.99  # Discount factor
epsilon = 1.0  # Exploration rate
epsilon_decay = 0.995  # Decay rate for exploration
min_epsilon = 0.01  # Minimum exploration rate
episodes = 1000  # Number of episodes to train

In [ ]:
episode_rewards = []
epsilon_values = []

In [ ]:
# Function to discretize the continuous state for MountainCar
def discretize_state(state):
    discrete_state = []
    for i in range(len(state)):
        scaling = (state[i] - state_bounds[i][0]) / (state_bounds[i][1] - state_bounds[i][0])
        new_state = int(scaling * num_buckets[i])
        new_state = min(num_buckets[i] - 1, max(0, new_state))  # Clip state within bucket range
        discrete_state.append(new_state)
    return tuple(discrete_state)

In [ ]:
# Initialize list to store frames for GIF
saved_frames = []

#### TODO: Implement the Q-Learning algorithm ####
# Q-Learning Algorithm for MountainCar with Frame Saving
for episode in range(episodes):
    state = env.reset()[0]
    state = discretize_state(state)
    done = False
    total_reward = 0

    while not done:
        #### TODO: Implement the epsilon-greedy action selection strategy ####
        # Replace the line below with the epsilon-greedy action selection logic.
        # Use `random.uniform(0, 1) < epsilon` to decide between exploring and exploiting.
        # When exploring, use `env.action_space.sample()` to select a random action.
        # When exploiting, select the action with the maximum Q-value using `np.argmax`.
        action = env.action_space.sample()  # Temporary default to prevent errors

        # Take action, observe new state, reward, done, truncated, and info
        next_state, reward, done, truncated, info = env.step(action)
        next_state = discretize_state(next_state)  # Discretize next state

        #### TODO: Implement the Q-table update using the Q-learning formula ####
        # Replace the line below with the Q-learning formula to update the Q-table.
        # Use `alpha`, `gamma`, `reward`, `next_state`, and `np.max()` for the update.
        q_table[state][action] = q_table[state][action]  # Temporary placeholder (no-op)

        # Update state and reward
        state = next_state
        total_reward += reward

        # Check if episode is done or truncated
        if done or truncated:
            break

    # Append total reward for each episode
    episode_rewards.append(total_reward)

    # Append current epsilon value
    epsilon_values.append(epsilon)

    # Epsilon decay
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

    #### TODO: Update the print from 100 to 1000 if your configured episodes exceed 10000 ####
    # Print results for every 100 episodes
    if episode % 100 == 0:
        print(f"Episode {episode} - Total Reward: {total_reward}")
        frame = env.render()  # Capture frame as image array
        saved_frames.append(frame)  # Save frame to list

# Close the environment after training
env.close()

In [ ]:
print("Training completed for MountainCar!")

Visualization of the results for MountainCar

In [ ]:
import imageio

# Save frames as a GIF
imageio.mimsave('mountain_car_training.gif', saved_frames, fps=15)
print("GIF saved as 'mountain_car_training.gif'")

In [ ]:
from IPython.display import Image

# Display the saved GIF
Image(filename='mountain_car_training.gif')

In [ ]:
plt.figure(figsize=(12, 5))

In [ ]:
# Total Reward per Episode
plt.subplot(1, 2, 1)
plt.plot(episode_rewards, color='red')
plt.title('Total Reward per Episode (MountainCar)')
plt.xlabel('Episode')
plt.ylabel('Total Reward')

In [ ]:
# Epsilon Decay over Episodes
plt.subplot(1, 2, 2)
plt.plot(epsilon_values, color='orange')
plt.title('Epsilon Decay over Episodes (MountainCar)')
plt.xlabel('Episode')
plt.ylabel('Epsilon Value')

In [ ]:
plt.tight_layout()
plt.show()

### TODO: Summarizing Epsilon Decay Observations

Now that you have implemented epsilon decay and visualized its impact, analyze your results by addressing the following questions:

1. **Epsilon Behavior**: How did epsilon change over time? Did it decrease as expected?
2. **Exploration vs. Exploitation**: How did the agent's behavior shift as epsilon decreased? Did it explore less and exploit more?
3. **Performance Impact**: How did different decay rates affect the agent's learning performance? Was there an optimal decay rate that balanced exploration and exploitation effectively?
4. **Real-World Implications**: If you were applying this in a real-world scenario (e.g., robotics, finance, or gaming), how would you justify your chosen epsilon decay strategy?

**Action:** Write a short analysis (at least 1-2 paragraphs) summarizing your observations in a markdown cell below.
